# Problem #1

In [1]:
import numpy as np

def l2(v):
    return np.sqrt(np.sum(v**2))

def stop_cond(A, x_new, x, lamb_new, lamb, tol):
    return abs(lamb_new - lamb) < tol

def power_method(A, x_0, tol=10**-12, max_iter=100, stop_cond=stop_cond):
    
    lamb = 0
    k = 1
    x = x_0 / l2(x_0)
    
    for i in range(max_iter):
        k += 1
        x_new = A @ x
        if l2(x_new) == 0:
            return lamb, x_0, i
        lamb_new = np.transpose(x_new) @ x
        
        # normalize x_new
        x_new = x_new / l2(x_new)
        
        if stop_cond(A, x_new, x, lamb_new, lamb, tol):
            return lamb_new, x_new, i
        
        lamb = lamb_new
        x = x_new
    
    raise Exception("No eigen value found in {} iterations".format(max_iter))

In [2]:
A = np.array([
    [ -3, -3, -2, -1,  0,  1],
    [ -3,  0, -1,  0,  1,  2],
    [ -2, -1,  3,  1,  2,  3],
    [ -1,  0,  1,  6,  3,  4],
    [  0,  1,  2,  3,  9,  5],
    [  1,  2,  3,  4,  5, 12]
], dtype="float64")

x_0 = np.array([1, 2, 3, 4, 5, 6])

r = power_method(A, x_0, tol=10**-8, max_iter=50)

print("{} total iterations.".format(r[2]))
print("Largest eigenvector of A is {}, with eigenvector:\n  v={}\n".format(*r))
print("A*v = lambda*v => {}, with tol={}".format(np.allclose(A @ r[1], r[0]*r[1], atol=10**-4), 10**-4))

9 total iterations.
Largest eigenvector of A is 18.87850316903313, with eigenvector:
  v=[-0.0172384   0.09572123  0.22290907  0.36719393  0.53227154  0.7229851 ]

A*v = lambda*v => True, with tol=0.0001


# Problem #2

Using the stop condition $\Vert A\vec{x} - \lambda\vec{x} \Vert < tol$:

In [3]:
shift_mu = 3.7
B = A - shift_mu * np.identity(A.shape[0])
B_inv = np.linalg.inv(B)

def stop_cond_alt(A, x_new, x, lamb_new, lamb, tol):
    return l2(A @ x_new - lamb_new * x_new) < tol

r = power_method(B_inv, x_0, tol=10**-8, max_iter=50, stop_cond=stop_cond_alt)

lamb = 1/r[0] + shift_mu
v = B_inv @ r[1]
v = v / l2(v)

print("{} total iterations.".format(r[2]))
print("Eigenvalue of A closest to {} is {} with eigenvector:\n  v={}\n".format(shift_mu, lamb, v))

32 total iterations.
Eigenvalue of A closest to 3.7 is 4.12214702965213 with eigenvector:
  v=[-0.11100169 -0.09131111 -0.03652607  0.91529329 -0.30140393 -0.22226251]



Using the stop condition from problem #1:

In [14]:
r = power_method(B_inv, x_0, tol=10**-8, max_iter=50, stop_cond=stop_cond)

lamb = 1/r[0] + shift_mu
v = B_inv @ r[1]
v = v / l2(v)

print("{} total iterations.".format(r[2]))
print("Eigenvalue of A closest to {} is {} with eigenvector:\n  v={}\n".format(shift_mu, lamb, v))

17 total iterations.
Eigenvalue of A closest to 3.7 is 4.122147030324733 with eigenvector:
  v=[-0.11099979 -0.09130878 -0.03653495  0.91529407 -0.3014028  -0.22226126]



# Problem #3

In [15]:
def LUwithP(A):
    
    size = A.shape[0]
    
    L = np.identity(size)
    U = np.copy(A)
    P = np.identity(size)
    
    # iterate over the columns/rows
    for j in range(size):
        # finds the index of largest pivot in column j, row j to size
     
        pi = np.argmax(np.abs(U[j:,j]))
      
        # swap rows in U
        U[[j, pi +j], j:] = U[[pi +j, j], j:]

        #swap rows in P
        P[[pi + j, j]] = P[[j, pi + j]]
        
        #swap rows in L
        if j > 0:
            L[[pi + j, j], :j] = L[[j, pi + j], :j]

        pivot = U[j][j]

        # iterate from next row to last row
        for i in range(j+1, size):

            # a - m * pivot = 0  =>  m = a/pivot            
            m = U[i][j] / pivot
            L[i][j] = m 
            
            # iterate through remaining columns in the row
            for k in range(j, size):
                U[i][k] = U[i][k] - m*U[j][k]
                
    return P,L,U

def forward_subs(L, b):
    
    size = L.shape[0]
    out = np.zeros(size)
    
    for k in range(size):
        
        out[k] = b[k] - np.sum([ (L[k][i] * out[i] )/ L[k][k] for i in range(k + 1) ])
        
    return out

def back_subs(U, b):
    size = U.shape[0]
    out = np.zeros(size)
    out[-1] = b[-1] / U[-1,-1]
    
    for k in range(size - 2, -1, -1):
        
        out[k] = (b[k] - np.sum([ (U[k][i] * out[i] ) for i in range(k+1, size) ])) / U[k][k]
  
    return out  

In [16]:
def inverse_iter_inv(A, x_0, shift_mu, tol=10**-10, max_iter=100, stop_cond=stop_cond):
    lamb = 0
    k = 1
    x = x_0 / l2(x_0)
    B = A - shift_mu * np.identity(A.shape[0])
    
    # Note using library inv function here
    B_inv = np.linalg.inv(B)
    
    for i in range(max_iter):
        k += 1
        x_new = B_inv @ x
        
        if l2(x_new) == 0:
            return 0, x_0, i
        
        lamb_new = np.transpose(x_new) @ x
        
        x_new = x_new / l2(x_new)
        if stop_cond(B_inv, x_new, x, lamb_new, lamb, tol):
            v = B_inv @ x_new
            return 1/lamb_new + shift_mu, v / l2(v), i
        lamb = lamb_new
        x = x_new
        
    raise Exception("No eigen value found in {} iterations".format(max_iter))

Checking against problem 2 result:

In [17]:
r = inverse_iter_inv(A, x_0, 3.7, tol=10**-8, max_iter=50, stop_cond=stop_cond_alt)
print("{} total iterations.".format(r[2]))
print("Eigenvalue of A closest to {} is {} with eigenvector:\nv={}\n".format(4.0, r[0], r[1]))

32 total iterations.
Eigenvalue of A closest to 4.0 is 4.12214702965213 with eigenvector:
v=[-0.11100169 -0.09131111 -0.03652607  0.91529329 -0.30140393 -0.22226251]



Using $\mu=4.0$ and same stop condition as problem #1:

In [18]:
r = inverse_iter_inv(A, x_0, 4.0, tol=10**-8, max_iter=50)
print("{} total iterations.".format(r[2]))
print("Eigenvalue of A closest to {} is {} with eigenvector:\nv={}\n".format(4.0, r[0], r[1]))

6 total iterations.
Eigenvalue of A closest to 4.0 is 4.122147029652752 with eigenvector:
v=[-0.11100169 -0.09131112 -0.03652605  0.91529328 -0.30140393 -0.22226252]



Using $\mu = 3.7$ and same stop condition as problem #1:

In [19]:
r = inverse_iter_inv(A, x_0, 3.7, tol=10**-8, max_iter=50)
print("{} total iterations.".format(r[2]))
print("Eigenvalue of A closest to {} is {} with eigenvector:\nv={}\n".format(3.7, r[0], r[1]))

17 total iterations.
Eigenvalue of A closest to 3.7 is 4.122147030324733 with eigenvector:
v=[-0.11099979 -0.09130878 -0.03653495  0.91529407 -0.3014028  -0.22226126]



Replacing `np.linalg.inv` with $LU$ factorization:

In [26]:
def inv(P, L, U, v):
    v_temp = forward_subs(L, P @ v)
    return back_subs(U, v_temp)

def rev(P, L, U, v):
    return P @ L @ U @ v

def inverse_iter_LU(A, x_0, shift_mu, tol=10**-10, max_iter=100):
    lamb = 0
    k = 1
    x = x_0 / l2(x_0)
    B = A - shift_mu * np.identity(A.shape[0])
    [P, L, U] = LUwithP(B)

    for i in range(max_iter):
        k += 1
     
        x_new = inv(P, L, U, x)
        
        if l2(x_new) == 0:
            return 0, x_0, i
        lamb_new = np.transpose(x_new) @ x
        x_new = x_new / l2(x_new)
        if abs(lamb_new - lamb) < tol:
            v = rev(P, L, U, x_new)
            return 1/lamb_new + shift_mu, x_new, i
        lamb = lamb_new
        x = x_new
        
    raise Exception("No eigen value found in {} iterations".format(max_iter))

Using $\mu = 4.0$:

In [27]:
r = inverse_iter_LU(A, x_0, 4.0, tol=10**-8, max_iter=50)
print("{} total iterations.".format(r[2]))
print("Eigenvalue of A closest to {} is {} with eigenvector:\nv={}\n".format(4.0, r[0], r[1]))

6 total iterations.
Eigenvalue of A closest to 4.0 is 4.122147029652752 with eigenvector:
v=[-0.11100164 -0.09131106 -0.03652629  0.91529331 -0.30140386 -0.22226251]



Using $\mu = 3.7$:

In [28]:
r = inverse_iter_LU(A, x_0, 3.7, tol=10**-8, max_iter=50)
print("{} total iterations.".format(r[2]))
print("Eigenvalue of A closest to {} is {} with eigenvector:\nv={}\n".format(4.0, r[0], r[1]))

17 total iterations.
Eigenvalue of A closest to 4.0 is 4.122147030324733 with eigenvector:
v=[-0.11100516 -0.09131538 -0.0365098   0.91529185 -0.301406   -0.2222648 ]



### Results

Comparing results from problem 2 and 3 with consistent stop conditions of problem 1 gives the following results:

|method|shift_mu|total iterations|
|------|---|---|
|`inverse + power_method` (prob. 2)|3.7|17|
|`inverse_iter_inv` (prob. 3)|3.7|17
|`inverse_iter_inv` (prob. 3)|4.0|6
|`inverse_iter_LU` (prob. 3)|3.7|17
|`inverse_iter_LU` (prob. 3)|4.0|6

Changing the shift to be closer to the eigenvector has the effect of reducing the number of itterations before the algorithm reaches the desired tolerance. At a high level the algorithm will return the closest eigenvalue to $\mu$ so adjusting $\mu$ to start closer to an eigenvalue will reduce the number of steps needed to reach that eighenvalue.
